### Choosing the Right Dataset
 customer service, twitter, slack, reddit, movie data

 https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

 whatever the data is we need to clean it and create sequence pairs, like dialog or sentence pairs

### Setting up the Bot

Just as we built a chatbot class to handle the methods for our rule-based and retrieval-based chatbots, we’ll build a chatbot class for our generative chatbot.

Inside, we’ll add a greeting method and a set of exit commands, just like we did for our closed-domain chatbots.

However, in this case, we’ll also import the seq2seq model we’ve built and trained on chat data for you, as well as other information we’ll need to generate a response.

As it happens, many cutting-edge chatbots blend a combination of rule-based, retrieval-based, and generative approaches in order to easily handle some intents using predefined responses and offload other inputs to a natural language generation system

In [ ]:
class ChatBot:
  
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")

  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
  
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on dialog from The Princess Bride. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    
    self.chat(user_response)
  
  def chat(self, reply):
    while not self.make_exit(reply):
      # change this line below:
      reply = input(self.generate_response(reply))
    
  # define .generate_response():
  def generate_response(self, user_input):
      encoder_states_value = encoder_model.predict(test_input)
      decoder_states_value = encoder_states_value
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, target_features_dict['<START>']] = 1.
      decoded_sentence = ''
  
      stop_condition = False
      while not stop_condition:
          # Run the decoder model to get possible 
          # output tokens (with probabilities) & states
              output_tokens, new_decoder_hidden_state, new_decoder_cell_state = decoder_model.predict([target_seq] + decoder_states_value)

            # Choose token with highest probability
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            # Use sampled_token_index to find our next word in reverse_target_features_dict. Save the result to sampled_token
            sampled_token = reverse_target_features_dict[sampled_token_index]
            decoded_sentence += ' ' + sampled_token
            # Exit condition: either hit max length
            # or find stop token.
            if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
              stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.

            # Update states
            decoder_states_value = [new_decoder_hidden_state, new_decoder_cell_state]

        return decoded_sentence
  
  
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
      
    return False
  
# instantiate your ChatBot below:
test = ChatBot()
response = test.generate_response('Hi')
print(response)


we will use the seq2seq model in the generate_response function. 
If we just paste our .generate_response() method from the seq2seq model it only works with preprocessed data that’s been converted into a NumPy matrix of one-hot vectors. That won’t do for our chatbot; we don’t just want to use test data for our output. We want the .generate_response() method to accept new user input.

Luckily, we can address this by building a method that translates user input into a NumPy matrix. Then we can call that method inside .generate_response() on our user inpu

Necessary changes:
1. 
In ChatBot, above .generate_response():- 

Define a new method called .string_to_matrix() with self and user_input as parameter- s.
Inside .string_to_matrix(), split up the user input string by setting tokens equal to re.findall(r"[\w']+|[^\s\w]", user_inpu- t).
Next, generate a NumPy matrix of zeros called user_input_mat```
rix:
user_input_matrix = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens),
  dtype='fl```o- at32')

Return the user_input_matrix NumPy matrix from th, t y again
2.
Between where you created user_input_matrix and where you returned it from .string_to_matrix(), iterate over each timestep (which is really the index) and token in enumerte(t`okens).

Set user_input_matrix[0, timestep, input_featur`es_dict[token]] to 1.. This creates a one-hot vector within the matrix for each word in 

3. At the top of the .generate_response() method, convert user_input into a NumPy matrix using self.string_to_matrix().
Pass the resulting user input matrix into encoder_model.predict() instead of the user_input strin

I need to create a py file with the necessary functions to import here, currently the cell below will not workg.the user input.t.

In [ ]:
import numpy as np
import re
from seq2seq import encoder_model, decoder_model, num_decoder_tokens, num_encoder_tokens, input_features_dict, target_features_dict, reverse_target_features_dict, max_decoder_seq_length, max_encoder_seq_length

class ChatBot:
  
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")

  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
  
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on dialog from The Princess Bride. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    
    self.chat(user_response)
  
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply))
    
  # define .string_to_matrix() below:
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype = "float32")
    for timestep, token in enumerate(tokens):
      user_input_matrix[0, timestep, input_features_dict[token]] = 1

    return user_input_matrix
  
  def generate_response(self, user_input):
    # change user_input into a NumPy matrix:
    input_matrix = self.string_to_matrix(user_input)
    # update argument for .predict():
    states_value = encoder_model.predict(input_matrix)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    chatbot_response = ''

    stop_condition = False
    while not stop_condition:
      output_tokens, hidden_state, cell_state = decoder_model.predict(
        [target_seq] + states_value)

      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      chatbot_response += " " + sampled_token
      
      if (sampled_token == '<END>' or len(chatbot_response) > max_decoder_seq_length):
        stop_condition = True
        
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.
      
      states_value = [hidden_state, cell_state]
      
    return chatbot_response
  
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
      
    return False
  
chatty_mcchatface = ChatBot()
# call .generate_response():
test = chatty_mcchatface.generate_response('What?')
print(test)



### Handling Unknown Words
Nice work! Our chatbot now knows how to accept user input. But there is a pretty large caveat here: our chatbot only knows the vocabulary from our training data. What if a user uses a word that the chatbot has never seen before?

With our current code, we’ll get a KeyError:

This is because in .string_to_matrix() we are looking for token in input_features_d```ict:

for timestep, token in enumerate(tokens):
  user_input_matrix[0, timestep, input_features_dict[token```]] = 1.

Currently, if the token doesn’t exist in the input_features_dict dictionary (which keeps track of all words in the training data), our program has no way of handling it.

Here are a few popular approaches to tackle unk- own words:

Tell the chatbot to ignore them, which is the simplest fix for smaller datasets, but can never generate those words as output. (Can you imagine scenarios when this could b- e a problem?)
Pause the chat process and have the chatbot ask what the entire utterance means. This requires the user to rephrase the entire utterance. This causes issues when working with a fairly limited dataset, since we may end up with the chatbot repeatedly asking the user to rephrase each in- put statement.
Add in a step for the chabot to register any unknown word as a '<UNK>' token. This is generally more complicated than the other two solutions. It would require that the training data is built out with '<UNK>' tokens and requires several ext



Because we’re using a tiny dataset to train our model, we’re going to just ignore words that haven’t been added into input_features_dict during training.

Fortunately, a simple if statement is all we need.

Add an if clause that checks that token is in input_features_dict so that a one-hot vector only gets created for a know:
`if token in input_features_dict.keys():if token in input_features_dict.keys():`


### Hooray! 
While our chatbot is far from perfect, it does produce innovative babble, allowing us to create a truly open-domain conversation. Just like with a machine translation model, we want to accommodate new sentence structures and creativity, something we can do with a generative model for our chatbot.

Of course, even with a better trained model, there are some issues that we need to consider.

First, there are ethical considerations that are much harder to discern and track when working with deep learning models. It’s very easy to accidentally train a model to reproduce latent biases within the training data.

We’re also making a pretty big assumption right now in our chatbot architecture: the chatbot responses only depend on the previous turn of user input. The seq2seq model we have here won’t account for any previous dialog. For example, our chatbot has no way to handle a simple back-and-forth like ```this:

User: “Do you have any siblings?”

Chatbot: “Yes, I do”

User: “How many?”
```
Chatbot: 🤯

In addition to topics that have been previously covered, the entire context of the chat is missing. Our chatbot doesn’t know anything about the user, their likes, their dislikes, etc.

As it happens, handling context and previously covered topics is an active area of research in NLP. Some proposed s- lutions include:

training the model to hang onto some previous num- ber of dialog turns
keeping track of the decoder’s hidden state-  across dialog turns
personalizing models by including user context during training or adding user context as it is incl

The final code is below.
I'll recreate it in VS Code with the functions in separate files.
I need to find a way to collect, clean and store the dialogs efficiently - sqlite?
I need to find a way to train the model incrementally
I need to find out how to add dialogs to the existing data and retrain the modeluded in the user input word:ra manual steps.

In [ ]:
import numpy as np
import re
from seq2seq import encoder_model, decoder_model, num_decoder_tokens, num_encoder_tokens, input_features_dict, target_features_dict, reverse_target_features_dict, max_decoder_seq_length, max_encoder_seq_length

class ChatBot:
  
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")

  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
  
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on dialog from The Princess Bride. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    
    self.chat(user_response)
  
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply))
    
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    states_value = encoder_model.predict(input_matrix)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    chatbot_response = ''

    stop_condition = False
    while not stop_condition:
      output_tokens, hidden_state, cell_state = decoder_model.predict(
        [target_seq] + states_value)
      
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      
      chatbot_response += " " + sampled_token
      
      if (sampled_token == '<END>' or len(chatbot_response) > max_decoder_seq_length):
        stop_condition = True
        
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.
      
      states_value = [hidden_state, cell_state]
      
    chatbot_response = chatbot_response.replace("<START>", "").replace("<END>", "")
      
    return chatbot_response
  
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
      
    return False
  
chatty_mcchatface = ChatBot()
chatty_mcchatface.start_chat()


